In [1]:
import os


os.chdir("../")

In [2]:
from dataclasses import dataclass
from pathlib import Path


@dataclass(frozen=True)
class BaseModelConfig:
    root_dir: Path
    base_model_path: Path
    updated_base_model_path: Path
    image_size: list
    learning_rate: float
    include_top: bool
    weights: str
    classes: int

In [3]:
from cnn_classifier.constants import *
from cnn_classifier.utils.common import read_yaml, create_directories


class ConfigurationManager:
    def __init__(
        self,
        config_file_path: Path = CONFIG_FILE_PATH,
        params_file_path: Path = PARAMS_FILE_PATH,
    ):
        self.config = read_yaml(config_file_path)
        self.params = read_yaml(params_file_path)

        create_directories([self.config.artifacts_root])

    def get_base_model_config(self) -> BaseModelConfig:
        cfg = self.config.prepare_base_model
        params = self.params.params

        create_directories([cfg.root_dir])

        base_model_config = BaseModelConfig(
            root_dir=cfg.root_dir,
            base_model_path=cfg.base_model_path,
            updated_base_model_path=cfg.updated_base_model_path,
            image_size=params.IMAGE_SIZE,
            learning_rate=params.LEARNING_RATE,
            include_top=params.INCLUDE_TOP,
            weights=params.WEIGHTS,
            classes=params.CLASSES,
        )

        return base_model_config

In [4]:
from typing import Optional
import tensorflow as tf


class PrepareBaseModel:
    def __init__(self, config: BaseModelConfig):
        self.config = config

    def get_base_model(self):
        self.model = tf.keras.applications.vgg16.VGG16(
            input_shape=self.config.image_size,
            weights=self.config.weights,
            include_top=self.config.include_top,
        )

        self.save_model(path=self.config.base_model_path, model=self.model)
        
    def update_base_model(self):
        self.full_model = self._prepare_full_model(
            model=self.model,
            classes=self.config.classes,
            freeze_all=True,
            freeze_till=None,
            learning_rate=self.config.learning_rate,
        )
        
        self.save_model(path=self.config.updated_base_model_path, model=self.full_model)

    @staticmethod
    def save_model(path: Path, model: tf.keras.Model):
        model.save(path)

    @staticmethod
    def _prepare_full_model(
        model: tf.keras.Model,
        classes: int,
        freeze_all: bool = True,
        freeze_till: Optional[int] = None,
        learning_rate: float = 0.01,
    ) -> tf.keras.Model:
        if freeze_all:
            model.trainable = False
        elif (freeze_till is not None) and (freeze_till > 0):
            for layer in model.layers[:-freeze_till]:
                layer.trainable = False

        flatten_in = tf.keras.layers.Flatten()(model.output)
        prediction = tf.keras.layers.Dense(units=classes, activation="softmax")(
            flatten_in
        )
        
        full_model = tf.keras.models.Model(inputs=model.input, outputs=prediction)
        full_model.compile(
            optimizer=tf.keras.optimizers.legacy.SGD(learning_rate=learning_rate),
            loss=tf.keras.losses.CategoricalCrossentropy(),
            metrics=["accuracy"],
        )
        full_model.summary()
        
        return full_model

In [5]:
from cnn_classifier import logger


class PrepareBaseModelPipeline:
    
    def run_pipeline(self):
        try:
            logger.info(f"Prepare base model started")
            configuration_manager = ConfigurationManager()
            base_model_config = configuration_manager.get_base_model_config()
            prepare_base_model = PrepareBaseModel(config=base_model_config)
            prepare_base_model.get_base_model()
            prepare_base_model.update_base_model()
            
        except Exception as e:
            logger.error(f"Prepare base model failed: {e}")
            raise e

In [6]:
prepare_base_model_pipeline = PrepareBaseModelPipeline()
prepare_base_model_pipeline.run_pipeline()

[ 2024-02-27 23:35:30,767 ] 8 2605349289 cnn_classifier -  INFO - Prepare base model started
[ 2024-02-27 23:35:30,774 ] 34 common cnn_classifier -  INFO - Loaded YAML file successfully from: config/config.yaml
[ 2024-02-27 23:35:30,777 ] 34 common cnn_classifier -  INFO - Loaded YAML file successfully from: params.yaml
[ 2024-02-27 23:35:30,779 ] 55 common cnn_classifier -  INFO - Created directory at: artifacts
[ 2024-02-27 23:35:30,780 ] 55 common cnn_classifier -  INFO - Created directory at: artifacts/prepare_base_model
Model: "model"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_1 (InputLayer)        [(None, 224, 224, 3)]     0         
                                                                 
 block1_conv1 (Conv2D)       (None, 224, 224, 64)      1792      
                                                                 
 block1_conv2 (Conv2D)       (None, 224, 224, 64)      369